In [0]:
import os

storage_account_name = "dota2lakehousenew"
storage_account_key = os.environ.get("AZURE_STORAGE_KEY")
container_name = "data"

spark.conf.set(
    f"fs.azure.account.key.{storage_account_name}.dfs.core.windows.net",
    storage_account_key
)

print("Storage account has been configured!")

Storage bağlantısı kuruldu!


In [0]:
gold_path = "abfss://data@dota2lakehousenew.dfs.core.windows.net/gold"

player_stats = spark.read.format("delta").load(f"{gold_path}/player_stats")
player_stats.createOrReplaceTempView("gold_player_stats")
print("gold_player_stats table has been created")

hero_stats = spark.read.format("delta").load(f"{gold_path}/hero_stats")
hero_stats.createOrReplaceTempView("gold_hero_stats")
print("gold_hero_stats table has been created")

daily_stats = spark.read.format("delta").load(f"{gold_path}/daily_stats")
daily_stats.createOrReplaceTempView("gold_daily_stats")
print("gold_daily_stats table has been created")

ml_features = spark.read.format("delta").load(f"{gold_path}/ml_features")
ml_features.createOrReplaceTempView("gold_ml_features")
print("gold_ml_features table has been created")

gold_player_stats tablosu olusturuldu
gold_hero_stats tablosu olusturuldu
gold_daily_stats tablosu olusturuldu
gold_ml_features tablosu olusturuldu


In [0]:
%sql
SELECT 
    account_id,
    total_kills, 
    total_deaths,
    total_assists,
    ROUND(avg_kda, 2) as kda_ratio
FROM 
    gold_player_stats
WHERE 
    total_matches > 5
ORDER BY 
    avg_kda DESC
LIMIT 10;

account_id,total_kills,total_deaths,total_assists,kda_ratio
2.02217968E8,49,16,49,12.27
1.16249155E8,68,44,193,11.97
1.73978074E8,70,25,99,11.79
3.20252024E8,68,12,62,11.76
1.32309493E8,40,12,48,11.75
3.92565237E8,53,22,100,11.67
2.21532774E8,60,24,127,11.18
1.18233883E8,52,24,78,10.95
3.61688848E8,177,58,180,10.77
2.20154695E8,54,22,57,10.63


In [0]:
%sql
SELECT 
    hero_id,
    times_played as total_picks,
    wins as total_wins,
    ROUND(win_rate * 100, 2) as win_rate_percentage
FROM 
    gold_hero_stats
ORDER BY 
    times_played DESC
LIMIT 10;

hero_id,total_picks,total_wins,win_rate_percentage
86.0,330,169,5121.0
100.0,299,151,5050.0
19.0,255,117,4588.0
9.0,248,121,4879.0
128.0,225,101,4489.0
83.0,210,114,5429.0
5.0,202,109,5396.0
25.0,191,98,5131.0
106.0,187,94,5027.0
51.0,179,94,5251.0


In [0]:
%sql
SELECT 
    match_date,
    match_count,
    ROUND(avg_duration / 60, 2) as avg_duration_minutes
FROM 
    gold_daily_stats
ORDER BY 
    match_date DESC
LIMIT 10;

match_date,match_count,avg_duration_minutes
2023-12-31,45,0.58
2023-12-30,49,0.57
2023-12-29,70,0.55
2023-12-28,74,0.54
2023-12-27,73,0.55
2023-12-26,71,0.58
2023-12-25,56,0.56
2023-12-24,67,0.54
2023-12-23,75,0.56
2023-12-22,60,0.58


In [0]:
%sql
SELECT 
    hero_id,
    ROUND(avg_gpm, 0) as avg_gpm,
    ROUND(avg_xpm, 0) as avg_xpm
FROM 
    gold_hero_stats
ORDER BY 
    avg_gpm DESC
LIMIT 10;

hero_id,avg_gpm,avg_xpm
109.0,701.0,710.0
89.0,674.0,640.0
53.0,666.0,707.0
25.0,655.0,783.0
46.0,653.0,803.0
95.0,649.0,751.0
94.0,643.0,719.0
12.0,641.0,731.0
69.0,640.0,691.0
113.0,637.0,682.0


In [0]:
%sql
SELECT 
    hero_id,
    ROUND(avg_kills, 2) as avg_kills,
    ROUND(avg_assists, 2) as avg_assists,
    times_played
FROM 
    gold_hero_stats
ORDER BY 
    avg_kills DESC
LIMIT 10;

hero_id,avg_kills,avg_assists,times_played
39.0,10.28,10.89,72
25.0,9.68,7.3,191
4.0,9.27,10.5,52
93.0,8.79,9.78,89
13.0,8.77,10.65,60
70.0,8.74,5.86,74
106.0,8.62,11.87,187
22.0,8.59,13.55,74
52.0,8.59,9.32,109
67.0,8.54,13.98,46
